This notebook implments batch downloading of Planet basemap quads of multiple months over pre-defined AOIs and produce a mosaic for each month. Only basemap quads intersecting the AOIs (not just their bounding boxes) will be downloaded. It requires your Planet API Key.

## Load packages

In [1]:
# import packages
import os
import json
import requests
from requests.auth import HTTPBasicAuth # import helper functions to make Basic request to Planet API
# import config  # needed if your Planet API key is put in a config file
import urllib.request
import geopandas as gpd
from shapely.geometry import box
import subprocess


/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


## Set parameters

In [2]:
# shapefile of AOIs
AOI_path='input_data/Rwanda_Boundary.shp'

# output folder to put in downloaded quads
out_folder = 'results'

# year and months to search for and download
str_year='2021'
# str_months=['02','04','06','08','10','12']
str_months=['03','04','08','10','11','12']

# product to download, i.e. 3-band RGB visual or 4-band analytic
product_prefix='planet_medres_visual_' # 3-band monthly visual mosaic
# product_prefix='planet_medres_normalized_analytic_' # 4-band monthly analytic mosaic

# country/region name
country='Rwanda'
# downloaded file name prefix
outname_prefix=country+'_'+product_prefix+str_year

# Planet base URL
API_URL = "https://api.planet.com/basemaps/v1/mosaics"

In [3]:
# get bbox from country boundary
AOIs=gpd.read_file(AOI_path).to_crs('epsg:4326')
AOIs

,ADM0_CODE,ADM0_NAME,CONTINENT,ISO3,ISO2,UNI,UNDP,FAOSTAT,GAUL,RIC_ISO3,REC_ISO3,HIH,geometry
0,205,Rwanda,Africa,RWA,RW,646.0,RWA,184.0,205.0,ICPAC-RWA,OTHER-RWA,1,"POLYGON ((30.46679 -1.06294, 30.46446 -1.06678..."


In [4]:
if not os.path.isdir(out_folder):
    os.makedirs(out_folder)

## Configuration

In [5]:
# try to get the API Key from the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

# otherwise pass in your API key if not exists
if PLANET_API_KEY is None:
    PLANET_API_KEY = '' # type in your API key here

#setup session
session = requests.Session()

#authenticate
# session.auth = (config.PLANET_API_KEY, "")
session.auth = (PLANET_API_KEY, "")

## Access and download quads intersecting AOIs

In [ ]:
file_names={str_month:[] for str_month in str_months} # list of output quad file names

In [ ]:
%%time
# loop through all months
for str_month in str_months:
    
    print('querying for month ',str_month)
    #set params for search using name of mosaic (can be found through Planet basemap viewer)
    parameters = {
        "name__is" :product_prefix+str_year+'-'+str_month+"_mosaic"
    }

    #make get request to access mosaic from basemaps API
    try:
        res = session.get(API_URL, params = parameters)
    except Exception as e: # reset connection
        print(e)
        print('resetting connection...')
        session = requests.Session()
        session.auth = (PLANET_API_KEY, "")
        res = session.get(API_URL, params = parameters)

    #response status code
    print('request status code: ',res.status_code)

    #print metadata for mosaic
    mosaic = res.json()
    # print('mosaic metedata:\n',json.dumps(mosaic, indent=2))

    #get mosaic id
    mosaic_id = mosaic['mosaics'][0]['id']
    print('mosaic ID: ',mosaic_id)

    #accessing quads using metadata from mosaic
    quads_url = "{}/{}/quads".format(API_URL, mosaic_id)

    # loop through all geometries
    for index, row in AOIs.iterrows():
        AOI=AOIs.iloc[[index]]
        # get bounding box of single geometry
        AOI_bbox=list(AOI.bounds.iloc[0])
        print('bounding box of geometry: ',AOI_bbox)

        #converting bbox to string for search params
        string_bbox = ','.join(map(str, AOI_bbox))

        #search for mosaic quad using AOI bbox string
        search_parameters = {
            'bbox': string_bbox,
            'minimal': True,
            '_page_size':5000 # IMPORTANT: need to set this (or the '_page'?) if your AOI contains more than 50 quads (the default limit), otherwise you will only get max of 50 records
        }

        #send request using the url and search parameters
        res = session.get(quads_url, params=search_parameters, stream=True)

        # return request results as json
        quads = res.json()

        # extract items, i.e. quads of mosaic
        items = quads['items']
        print('Number of quads from query: ',len(items))

        #iterate over quads and download to a folder
        for i in items:
            # only download the quads intersecting the AOI instead of just bbox
            quad_bbox = i["bbox"]
            quad_geom=box(*quad_bbox)
            if quad_geom.intersects(AOI.geometry.iloc[0]):

                # url link of each quad
                link = i['_links']['download']

                # set output file name
                name =outname_prefix+'_'+str_month+'_id_'+i['id']+ '.tif'
                filename = os.path.join(out_folder, name)
                if filename not in file_names[str_month]:
                    file_names[str_month].append(filename)
                # download file if not existing
                if not os.path.isfile(filename):
                    print('downloading quad as',filename)
                    urllib.request.urlretrieve(link, filename)
                else:
                    print('quad file exists')
            else:
                print('quad within AOI bounding box but not intersecting AOI')

## Mosaic all quads for each month

In [29]:
outnames_mosaic=[]
for str_month in str_months:
    print('mosaic all quads for month ',str_month)
    outname_mosaic=outname_prefix+'_'+str_month+'_mosaic.tif'
    outname_mosaic=os.path.join(out_folder, outname_mosaic)
    outnames_mosaic.append(outname_mosaic)
    if os.path.exists(outname_mosaic):
        print('mosaic file exists')
    else:
        print('mosaic to file ',outname_mosaic)
        quad_files=[fn for fn in file_names[str_month]]
        cmd=['gdal_merge.py','-co','COMPRESS=DEFLATE','-co','BIGTIFF=IF_SAFER','-o',outname_mosaic]
        cmd.extend(quad_file for quad_file in quad_files)
        # using gdal to merge
        subprocess.run(cmd)

mosaicking all quads for month  03
mosaicking to file  results/Rwanda_planet_medres_visual_2021_03_mosaic.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
mosaicking all quads for month  04
mosaicking to file  results/Rwanda_planet_medres_visual_2021_04_mosaic.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
mosaicking all quads for month  08
mosaicking to file  results/Rwanda_planet_medres_visual_2021_08_mosaic.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
mosaicking all quads for month  10
mosaic file exists
mosaicking all quads for month  11
mosaic file exists
mosaicking all quads for month  12
mosaic file exists


## Delete quad files to save space (optional)

In [30]:
for str_month in str_months:
    quad_files=file_names[str_month]
    for fn in quad_files:
        if os.path.exists(fn):
            os.remove(fn)

## Clip mosaics to buffered AOIs, extract RGB bands and delete unclipped mosaics (to save space)

In [32]:
buffer_dist=1300 # make slightly larger than a chunk size (e.g. 256 pixels) to ensure no pixels in chunks overlapping the AOI will be clipped out
buffered_AOI=AOIs.to_crs('epsg:3857').buffer(buffer_dist)
buffered_AOI_path=os.path.basename(AOI_path)[:-4]+'_buffered_'+str(buffer_dist)+'m.shp'
buffered_AOI_path=os.path.join(out_folder,buffered_AOI_path)
buffered_AOI.to_file(buffered_AOI_path)

In [39]:
for mosaic in outnames_mosaic:
    print('clipping',mosaic)
    outname=mosaic[:-4]+'_clipped.tif'
    if os.path.exists(outname):
        print('clipped mosaic file exists')
    else:
        # clip to AOI
        outname_temp=mosaic[:-4]+'_intermediate.tif'
        gdal_cmd=["gdalwarp", "-of", "GTiff", '-cutline',buffered_AOI_path,'-crop_to_cutline',
                  '-co','COMPRESS=DEFLATE',mosaic,outname_temp]
        p1=subprocess.run(gdal_cmd)
        # extract bands and compress
        gdal_cmd=["gdal_translate", "-of", "GTiff", '-b','1','-b','2','-b','3','-co','COMPRESS=DEFLATE',outname_temp,outname]
        p2=subprocess.run(gdal_cmd)
        if (p1.returncode==0)and(p2.returncode==0):
            # delete intermediate mosaic
            os.remove(outname_temp)
            # delete unclipped mosaic
            os.remove(mosaic)

clipping results/Rwanda_planet_medres_visual_2021_03_mosaic.tif
Using band 4 of source image as alpha.
Creating output file that is 48027P x 42325L.
Processing results/Rwanda_planet_medres_visual_2021_03_mosaic.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 48027, 42325
0...10...20...30...40...50...60...70...80...90...100 - done.
clipping results/Rwanda_planet_medres_visual_2021_04_mosaic.tif
Using band 4 of source image as alpha.
Creating output file that is 48027P x 42325L.
Processing results/Rwanda_planet_medres_visual_2021_04_mosaic.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 48027, 42325
0...10...20...30...40...50...60...70...80...90...100 - done.
clipping results/Rwanda_planet_medres_visual_2021_08_mosaic.tif
Using band 4 of source image as alpha.
Creating output file that is 48027P x 42325L.
Processing results/Rwanda_planet_medres_visual_2021_08_mosaic.tif [1/1] : 0...10...20...30...40..